## Import modules

In [1]:
import os
import pandas as pd
import numpy as np

import datetime
from dateutil.relativedelta import relativedelta

import seaborn as sns
from bokeh.plotting import figure, show, output_notebook
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm  

from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

os.chdir('/Users/Evan/Desktop/TB/TB_Nation/')
files = os.listdir()

## Getting and cleaning data

In [2]:
datasets = pd.DataFrame()
for i in range(0,len(files)):
    for j in os.listdir(files[i]):
        data = pd.read_excel(files[i]+'/'+ j ,skiprows=1).iloc[:1,0:5]
        data = data.rename(columns={'Unnamed: 0':'Area','发病数':'Incidence','死亡数':'Death','发病率':'Incidence_rate','死亡率':'Death_rate'})
        data['Year'],data['Month'],data['Day'] = files[i],j[4:6],'01'
        datasets = pd.concat([datasets,data])
# datasets.index=range(0,len(datasets))
datasets['Date'] = datasets['Year'] + datasets['Month'] + datasets['Day']
datasets.index = pd.to_datetime(datasets['Date'])
datasets = datasets.drop(['Year','Month','Day','Date'],axis=1)
datasets.head()

,Area,Incidence,Death,Incidence_rate,Death_rate
Date,,,,,
2004-01-01,全 国,99466,67,7.651937,0.005154
2004-02-01,全 国,84156,68,6.474136,0.005231
2004-03-01,全 国,94360,111,7.259132,0.008539
2004-04-01,全 国,91944,121,7.073268,0.009309
2004-05-01,全 国,90379,151,6.952873,0.011616


In [4]:
datasets.to_excel('/Users/Evan/Desktop/TB_nation.xlsx')

## Exploratory Data analysis

In [ ]:
datasets.Incidence_rate.plot(figsize=(12,8), title= 'Monthly TB Incidence Rate', fontsize=14)
# plt.savefig('month_TB.png', bbox_inches='tight')

In [ ]:
datasets_pred = datasets[datasets.index>='2014-01-1']
datasets = datasets[datasets.index<'2014-01-01']
datasets.shape

In [ ]:
decomposition = seasonal_decompose(datasets.Incidence_rate,freq=12)

In [ ]:
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(18,8)

## SARIMA

### Test of stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
#     rolmean = pd.rolling_mean(timeseries, window=12)
    rolmean = timeseries.rolling(window=12,center=False).mean()
#     rolstd = pd.rolling_std(timeseries, window=12)
    rolstd = timeseries.rolling(window=12,center=False).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='BIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(datasets.Incidence_rate)

In [ ]:
test_stationarity(datasets['Incidence_rate'])

### First difference

In [ ]:
datasets['first_diff'] = datasets.Incidence_rate - datasets.Incidence_rate.shift(1)
test_stationarity(datasets.first_diff.dropna(inplace=False))

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(datasets.first_diff.iloc[1:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(datasets.first_diff.iloc[1:], lags=40, ax=ax2)

### Seasonal difference

In [ ]:
datasets['seasonal_difference'] = datasets.Incidence_rate - datasets.Incidence_rate.shift(12)  
test_stationarity(datasets.seasonal_difference.dropna(inplace=False))

### Seasonal first difference 

In [ ]:
datasets['seasonal_first_difference'] = datasets.first_diff-datasets.first_diff.shift(12)
test_stationarity(datasets.seasonal_first_difference.dropna(inplace=False))

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(datasets.seasonal_first_difference.iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(datasets.seasonal_first_difference.iloc[13:], lags=40, ax=ax2)

## Bulid Model

In [ ]:
mod = sm.tsa.SARIMAX(datasets.Incidence_rate, trend='n', order=(0,1,2), seasonal_order=(0,1,1,12))
results = mod.fit()
print(results.summary())

In [ ]:
datasets['forecast'] = results.predict(start = 13, end= 131, dynamic= False)  
datasets[['Incidence_rate', 'forecast']].plot(figsize=(12, 8))

In [ ]:
npredict =datasets.Incidence_rate['2004'].shape[0]
fig, ax = plt.subplots(figsize=(12,6))
npre = 12
ax.set(title='National TB', xlabel='Date', ylabel='Riders')
ax.plot(datasets.index[-npredict-npre+1:], datasets.ix[-npredict-npre+1:, 'Incidence_rate'], 'o', label='Observed')
ax.plot(datasets.index[-npredict-npre+1:], datasets.ix[-npredict-npre+1:, 'forecast'], 'g', label='Dynamic forecast')
legend = ax.legend(loc='lower right')
legend.get_frame().set_facecolor('w')

In [ ]:
dta = pd.concat([datasets, datasets_pred])
dta.tail(13)

In [ ]:
dta['forecast'] = results.predict(start=13,end=131,dynamic=False)
dta['forecast'].tail(13)

In [ ]:
output_notebook()
p = figure(x_axis_type="datetime", title='Nation TB Incidence Rate in China')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=.5
p.line(dta.index, dta['Incidence_rate'], line_color="gray", line_dash="4 4", line_width=1)
p.circle(dta.index, dta['Incidence_rate'], size=6, color='olivedrab')
show(p)

In [ ]:
output_notebook()
pred = dta[dta.index>='2014-01-01']
p = figure(x_axis_type="datetime", title='Nation TB Incidence Rate in China')
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=.5
p.line(dta.index, dta['Incidence_rate'], line_color="gray", line_dash="4 4", line_width=1)
p.circle(dta.index, dta['Incidence_rate'], size=6, color='olivedrab')
p.scatter(pred.index,pred['forecast'],size=6,color='navy')
show(p)

In [ ]:
dta.tail(13)

In [ ]:
# dta.to_csv('/Users/Evan/Desktop/Nation TB Monthly.csv',encoding='GB2312')

In [ ]:
dta['forecast'] = results.predict(start = 13, end= 131, dynamic= False)  
dta[['Incidence_rate','forecast']].plot(figsize=(12, 8))

## SARIMA-GRNN Hybrid Model

## Load Packages

In [ ]:
from sklearn import preprocessing as pp
from sklearn import cross_validation as cv
from neupy.algorithms import GRNN as grnn
from neupy.estimators import mse

In [ ]:
data_train = dta.loc['2005-02-1':'2014-12-01'][['Incidence_rate','forecast']]
x_train = data_train.loc[:'2013-12-01']['forecast']
y_train = data_train.loc[:'2013-12-01']['Incidence_rate']
x_test = data_train.loc['2014-01-01':]['forecast']
y_test = data_train['2014-01-01':]['Incidence_rate']

In [ ]:
data_train

In [ ]:
def try_std(x):
    nn = grnn(std =x ,verbose=False)
    nn.train(x_train,y_train)
    y_pred = nn.predict(x_test)
    print(mse(y_pred,y_test))

In [ ]:
for x in np.linspace(0.05,1,20):
        print(x)
        try_std(x)
        print('--\n')

In [ ]:
mod_GRNN = grnn(std = 0.108 ,verbose=False)
mod_GRNN.train(x_train,y_train)
data_train.loc[:,'NN']=mod_GRNN.predict(data_train.loc[:,'forecast'])

In [ ]:
data_train[['Incidence_rate','forecast','NN']].plot(figsize=(12, 8)) 

In [ ]:
start = datetime.datetime.strptime("2015-01-01", "%Y-%m-%d")
date_list = [start + relativedelta(months=x) for x in range(0,132)]
future = pd.DataFrame(index=date_list, columns= dta.columns)
TB_future = pd.concat([dta, future])
TB_future['forecast'] = results.predict(start = 120, end = 263, dynamic= False)  
TB_future['NN'] = mod_GRNN.predict(TB_future['forecast'])
TB_future[['Incidence_rate', 'forecast','NN']].plot(figsize=(12, 8)) 

## Predict 2015-2025

In [ ]:
TB_2025 = TB_future[TB_future.index>='2025-01-01']['forecast']

In [ ]:
np.sum(TB_2025)